In [1]:
import os
import glob
import pandas as pd
import requests
#import validators
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from pathlib import Path
from zipfile import ZipFile
from os.path import exists

In [2]:
# Choose what to import
coinpair_list = ['SOLBUSD','BTCBUSD','ETHBUSD']
#coinpair_list = ['ADABUSD','SOLBUSD']
binance_timeframes_list = ['1m', '5m', '30m', '1h', '1d']
#TODO: Add coinpair to list

In [3]:
# Load Environment Variables
load_dotenv()

# Gets MongoDB Connection String
MDB_CONNECTION_STRING = os.getenv('MDB_CONNECTION_STRING')

# Function to connect to the Mongo DB
def get_database():
    try:
        client = MongoClient(MDB_CONNECTION_STRING)
        db = client["project-02"]
        return db
    except Exception as e:
        print(e)
        
# Connect to the db
db = get_database()

# Test Connection
serverStatusResult=db.command("serverStatus")
print(serverStatusResult["version"])

5.0.6


In [4]:
def get_coinpairs():
    # Get the coinpairs from the Database
    db_coinpairs = db["coinpairs"].find({"exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = list(binance_coinpairs_df['pair'])
    return coinpair_list
#coinpair_list = get_coinpairs()
#print(coinpair_list)

In [5]:
def get_timeframes():
    # Get the timeframes from the Database
    db_binance_timeframes = db["binance_timeframes"].find()

    # convert the dictionary objects to dataframe
    binance_timeframes_df = pd.DataFrame(db_binance_timeframes)

    # see the magic
    binance_timeframes_list = list(binance_timeframes_df['timeframe'])
    return binance_timeframes_list
#binance_timeframes_list = get_timeframes()
#print(binance_timeframes_list)

In [6]:
# Validates for if the remote file exists
def url_exists(url):
    if not url:
        raise ValueError("url is required")
    try:
        resp = requests.head(url)
        return True if resp.status_code == 200 else False
    except Exception as e:
        return False

In [7]:
# TODO: REMOVE FOLDERS AUTOMATICALLY
def process_binance_files(timeframe, coinpair, binance_timeframes_list, coinpair_list):
    from datetime import datetime
    max_years = 5
    current_year = datetime.now().year
    last_month = datetime.now().month - 1
    year = current_year
    months = 12
    month = last_month
    while (month <= months) and (year >= (current_year-max_years)):
        str_month = str(month)
        if len(str_month)==1:
            str_month = "0" + str(month)
        else:
            str_month = str(month)
        directory = "./"+coinpair+"_"+timeframe
        filename = coinpair+'-'+timeframe+'-'+str(year)+'-'+str_month+'.zip'
        csv_filename = coinpair+'-'+timeframe+'-'+str(year)+'-'+str_month+'.csv'
        # Check if file exists if it does, skip.
        file_exists = exists(directory+"/"+csv_filename)
        url = 'https://data.binance.vision/data/spot/monthly/klines/'+coinpair+'/'+timeframe+'/'+filename
        print(url)
        valid=url_exists(url)
        if valid==True:
            if file_exists == False:
                r = requests.get(url, allow_redirects=True)
                if not os.path.exists(directory):
                    os.makedirs(directory)
                location = directory +"/"+filename
                open(location, 'wb').write(r.content)
                os.chdir(coinpair+"_"+timeframe)
                with ZipFile(filename, 'r') as zipObj:
                    # Extract all the contents of zip file in current directory
                    zipObj.extractall()
                    if os.path.exists(filename):
                        os.remove(filename)
                os.chdir("..")
            else:
                print(f"File {csv_filename} exists, skipping download..")
            # Load csv and upload to mongodb
            csv_to_mongo(directory+"/"+csv_filename, coinpair+"_"+timeframe)
        else:
            year = year - max_years
        # Update month and year
        if month==1:
            month = 12
            year = year-1
        else:
            month = month - 1
        

In [8]:
def csv_to_mongo(csv_filename, collection_name):
    headers=["open_time", "open", "high", "low","close","volume","close_time","quote_asset_volume","numer_trades","taker_base_volume","taker_quote_volume","ignore"]
    kline_df = pd.read_csv(Path(csv_filename), parse_dates = True, infer_datetime_format = True, names=headers)
    kline_df['open_time'] = kline_df['open_time'].values.astype(dtype='datetime64[ms]')
    kline_df['close_time'] = kline_df['close_time'].values.astype(dtype='datetime64[ms]')
    kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]] = kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]].astype(float)
    kline_df = kline_df.drop(columns=['ignore'])
    kline_dict = kline_df.to_dict("records")
    db[collection_name].insert_many(kline_dict)
    db[collection_name].create_index([ ("open_time", -1) ])
    

In [9]:
def process(binance_timeframes_list, coinpair_list):
    for timeframe in binance_timeframes_list:
        for pair in coinpair_list:
            collection_name=pair+"_"+timeframe
            if collection_name not in db.list_collection_names():  
                print(f"Creating collection: {collection_name}")
                process_binance_files(timeframe, pair, binance_timeframes_list, coinpair_list)
            else:
                print(f"Collection {collection_name} found, skipping..")

In [10]:
process(binance_timeframes_list, coinpair_list)

Collection SOLBUSD_1m found, skipping..
Creating collection: BTCBUSD_1m
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2022-03.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2022-02.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2022-01.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2021-12.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2021-11.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2021-10.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2021-09.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2021-08.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2021-07.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBUSD/1m/BTCBUSD-1m-2021-06.zip
https://data.binance.vision/data/spot/monthly/klines/BTCBU

In [11]:
#TODO: ADD COINS PAIRS LIST TO THE DB

In [12]:
# Function to Get the Fear and Greed Index from  https://alternative.me
def get_fng():
    url = 'https://api.alternative.me/fng/?limit='+str(365*10)
    valid=url_exists(url)
    fng=requests.get(url).json()
    fng_data_df = pd.DataFrame(fng["data"])
    fng_data_df['timestamp'] = pd.to_datetime(fng_data_df['timestamp'], unit='s')
    fng_data_df = fng_data_df.drop(columns=['time_until_update'])
    fng_data_df['value'] = fng_data_df['value'].astype(int)
    print(fng_data_df)
    return fng_data_df

# Function to Update the Fear and Greed Index into our MondoDB
def update_fng(fng_data_df):
    collection_name = "fear_greed_index"
    data = fng_data_df.to_dict("records")
    db[collection_name].drop()
    db[collection_name].insert_many(data)
    db[collection_name].create_index([ ("timestamp", -1) ])
    print("Done downloading Fear and Greed Index")

In [13]:
fng_data_df = get_fng()
update_fng(fng_data_df)

      value value_classification  timestamp
0        52              Neutral 2022-04-04
1        48              Neutral 2022-04-03
2        52              Neutral 2022-04-02
3        50              Neutral 2022-04-01
4        52              Neutral 2022-03-31
...     ...                  ...        ...
1516     11         Extreme Fear 2018-02-05
1517     24         Extreme Fear 2018-02-04
1518     40                 Fear 2018-02-03
1519     15         Extreme Fear 2018-02-02
1520     30                 Fear 2018-02-01

[1521 rows x 3 columns]
Done downloading Fear and Greed Index
